In [1]:
import pandas as pd
import numpy as np
from run_autosklearn import AutoML


breast_data = pd.read_csv('missing_datasets/breast.csv')
Diabetes_data = pd.read_csv('missing_datasets/diabetes.csv')


/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
breast_data.drop(columns=['subspaces'],inplace=True)
breast_data.head(5)

,d1,d2,d3,d4,d5,d6,d7,d8,d9,is_anomaly
0,1,1,1,3,2,1,1,1,1,0
1,3,1,1,1,2,1,2,1,1,0
2,2,1,1,1,2,1,2,1,1,0
3,2,1,1,1,2,1,2,1,1,0
4,7,6,10,5,3,10,9,10,2,0


In [3]:
Diabetes_data.drop(columns=['subspaces'],inplace=True)
Diabetes_data.head(5)

,d1,d2,d3,d4,d5,d6,d7,d8,is_anomaly
0,2,75,64,24,55,29.7,0.370,33,0
1,2,120,76,37,105,39.7,0.215,29,0
2,13,129,0,30,0,39.9,0.569,44,0
3,9,72,78,25,0,31.6,0.280,38,0
4,11,155,76,28,150,33.3,1.350,51,0


Insert missing values to both datasets!

In [4]:
from generate_missing_values import generate_mcar_missing_values

# Find the feature's names.
diabetes_cols = [i for i in list(Diabetes_data.columns) if i != 'is_anomaly']
breast_cols = [i for i in list(breast_data.columns) if i != 'is_anomaly']

#Only inser missing data at features!
Diabetes_data.loc[:,diabetes_cols] = generate_mcar_missing_values(Diabetes_data.loc[:,diabetes_cols], 0.25).copy()

breast_data.loc[:,breast_cols] = generate_mcar_missing_values(breast_data.loc[:,breast_cols], 0.25).copy()

Running AutoSklearn with out own imputation methods on Diabetes dataset

In [5]:
AutoML(Diabetes_data.loc[:,diabetes_cols].to_numpy(), Diabetes_data.loc[:,'is_anomaly'].to_numpy())

[WARNING] [2023-11-10 23:17:29,403:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-11-10 23:19:35,028:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-11-10 23:20:33,009:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-11-10 23:20:45,457:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-11-10 23:21:49,849:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-11-10 23:21:58,980:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WAR

--- Logging error ---
Traceback (most recent call last):
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/logging/__init__.py", line 1089, in emit
    self.flush()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/logging/__init__.py", line 1069, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/threading.py", line 890, in _bootstrap
    self._bootstrap_inner()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/socketserver.py", line 683, in process_request_thread
    self.finish_request(request, client_address)
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestH

[ERROR] [2023-11-11 01:02:35,138:Client-AutoML(1):8734c4c7-800e-11ee-9aa1-cfd44a64255d] [Errno 28] No space left on device
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 316, in run
    self._incorporate_run_results(run_info, result, time_left)
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 563, in _incorporate_run_results
    self.save()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/smac/optimizer/smbo.py", line 575, in save
    self.runhistory.save_json(fn=os.path.join(output_dir, "runhistory.json"))
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8

--- Logging error ---
Traceback (most recent call last):
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/logging/__init__.py", line 1089, in emit
    self.flush()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/logging/__init__.py", line 1069, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/site-packages/ipykernel/kernelapp.py

OSError: [Errno 28] No space left on device

Running AutoSklearn with out own imputation methods on Breast dataset

In [ ]:
AutoML(breast_data.loc[:,breast_cols].to_numpy(), breast_data.loc[:,'is_anomaly'].to_numpy())

auto-sklearn results:
  Dataset name: d9b70b62-7fee-11ee-bdd0-cfd44a64255d
  Metric: accu
  Best validation score: 0.951359
  Number of target algorithm runs: 22
  Number of successful target algorithm runs: 17
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

Lowest loss: 0.04864091559370532
Best configuration: Configuration(values={
  'balancing:strategy': 'none',
  'classifier:__choice__': 'random_forest',
  'classifier:random_forest:bootstrap': 'True',
  'classifier:random_forest:criterion': 'gini',
  'classifier:random_forest:max_depth': 'None',
  'classifier:random_forest:max_features': 0.5,
  'classifier:random_forest:max_leaf_nodes': 'None',
  'classifier:random_forest:min_impurity_decrease': 0.0,
  'classifier:random_forest:min_samples_leaf': 1,
  'classifier:random_forest:min_samples_split': 2,
  'classifier:random_forest:min_weight_fraction_leaf': 0.0,
  